# 💻 Reklassifisering av data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GMGI221-2024/forelesninger/blob/main/09_reklassifisering.ipynb)

Reklassifisering av data basert på spesifikke kriterier er en vanlig oppgave når man utfører GIS
analyse. Formålet med denne notebooken er å se hvordan vi kan reklassifisere verdier
basert på noen kriterier. Man kunne for eksempel klassifisere informasjon basert på
reisetider og boligpriser ved hjelp av disse kriteriene:

1. Hvis reisetiden til jobben min er mindre enn 30 minutter, **OG**
2. husleien for leiligheten er mindre enn 10000 kr per måned

Hvis begge kriteriene er oppfylt: Jeg går for å se leiligheten og prøver å leie den
Hvis ikke: Jeg fortsetter å lete etter noe annet


I denne opplæringen vil vi:

1. Bruk klassifiseringsskjemaer fra PySAL [mapclassify
   bibliotek](https://pysal.org/mapclassify/) for å klassifisere befolkningstall til
   flere klasser.

2. Opprett en egendefinert klassifisering for å klassifisere og benytte den


## Inputdata

Vi vil bruke befolkningsdata fra [SSB](https://kart.ssb.no/) for Oslo, som inneholder 2380 celler for Oslo kommune.

In [1]:
import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_MAPPE = NOTEBOOK_PATH / "data"

In [ ]:
import geopandas

rutenett = geopandas.read_file(DATA_MAPPE / "ssb_rutenett" / )



## Vanlige klassifiseringer

### Klassifiseringsskjemaer for tematiske kart

[PySAL](https://pysal.org/) -modulen er et omfattende Python-bibliotek for romlig
analyse. Det inkluderer også alle de vanligste dataklassifiseringene som er
brukt vanlig f.eks. når man visualiserer data. Tilgjengelige kartklassifiseringer i [pysal's
mapclassify -modul](https://github.com/pysal/mapclassify):

- Box Plot
- Equal Interval
- Fisher Jenks
- Fisher Jenks Sampled
- HeadTail Breaks
- Jenks Caspall
- Jenks Caspall Forced
- Jenks Caspall Sampled
- Max P Classifier
- Maximum Breaks
- Natural Breaks
- Quantiles
- Percentiles
- Std Mean
- User Defined



**NoData-verdiene presenteres med verdi -1**. 
Derfor må vi først fjerne No Data-verdiene.

In [3]:
# Inkluder bare data som er over eller lik 0


La oss plotte dataene og se hvordan det ser ut
- `cmap` parameter definerer fargekartet. Les mer om [valg av fargekart i matplotlib](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html)
- `scheme` alternativ skalerer fargene i henhold til et klassifiseringsskjema (krever at `mapclassify` modulen er installert):

In [ ]:
# Plott ved hjelp av 9 klasser og klassifiser verdiene ved hjelp av "Natural Breaks" klassifisering


Som vi kan se fra dette kartet, er befolkningen større i sentrum, men det er også noen områder med få beboere i
noen andre områder (hvor fargen er rød).

### Bruk av klassifiseringer på data

Som nevnt, definerer `scheme` alternativet klassifiseringsskjemaet ved hjelp av
`pysal/mapclassify`. La oss se nærmere på hvordan disse klassifiseringene fungerer.

#### Natural Breaks

#### Quantiles (standard er 5 klasser):

#### Trekk ut terskelverdiene

Det er mulig å trekke ut terskelverdiene i en matrise:

La oss bruke en av `Pysal` klassifiseringene på dataene våre og klassifisere
befolkningen til 9 klasser
Klassifiseringen må først initialiseres med `make()` funksjonen som tar
antall ønskede klasser som inngangsparameter

In [9]:
# Lag en Natural Breaks klassifisering


- Nå kan vi bruke klassifiseringen på dataene våre ved å bruke `apply` -funksjonen

In [ ]:
# Klassifiser dataene



In [ ]:
type(klassifiseringer)

Ok, så nå har vi en DataFrame der inngangskolonnen vår ble klassifisert til 9
forskjellige klasser (tallene 1-9) basert på [Natural Breaks
klassifisering](http://wiki-1-1930356585.us-east-1.elb.amazonaws.com/wiki/index.php/Jenks_Natural_Breaks_Classification).

Vi kan også legge til klassifiseringsverdiene direkte i en ny kolonne i vår dataframe:

In [ ]:
# Gi nytt navn til kolonnen så vi vet at den ble klassifisert med naturlige brudd


# Sjekk de opprinnelige verdiene og klassifiseringen


Flott, nå har vi disse verdiene i GeoDataFramen. La oss
visualisere resultatene og se hvordan de ser ut.

In [ ]:
# Plott


Og her går vi, nå har vi et kart der vi har brukt en av de vanlige
klassifiseringene for å klassifisere dataene våre i 9 klasser.


## Plott et histogram

Et histogram er en grafisk representasjon av datafordelingen. Når
man klassifiserer data, er det alltid bra å vurdere hvordan dataene er fordelt,
og hvordan klassifiseringsskjemaet deler verdier i forskjellige områder.

- plott histogrammet ved hjelp av [pandas.DataFrame.plot.hist](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.hist.html)
- Antall histogrambinner (grupper av data) kan kontrolleres ved hjelp av parameteren `bins`:

In [ ]:
# Histogram for befolkningsrutenettet


La oss også legge til terskelverdier på toppen av histogrammet som vertikale linjer.

- Natural Breaks:

In [ ]:
import matplotlib.pyplot as plt

# Definer klassifiserer


# Plott histogram for befolkningsrutenettet


# Legg til vertikale linjer for klassebrudd


- Kvartiler:

In [ ]:
# Definer klassifiserer


# Plott histogram for befolkningsrutenettet


## Bruke en egendefinert klassifisering

### Multikriterium dataklassifisering

La oss klassifisere geometriene i to klasser basert
på en gitt `terskel` -parameter. Hvis området til en polygon er lavere enn terskelverdien, vil utgangskolonnen få en verdi
0, hvis den er større, vil den få en verdi 1. Denne typen klassifisering kalles ofte en [binær
klassifisering](https://en.wikipedia.org/wiki/Binary_classification).

For å klassifisere hver rad i vår GeoDataFrame kan vi iterere over hver rad, eller vi kan bruke
en funksjon for hver rad. I vårt tilfelle vil vi bruke en lambda funksjon for hver rad i
vår GeoDataFrame, som returnerer en verdi basert på de vilkårene vi gir.

La oss klassifisere basert på to kriterier: og finne ut rutenettceller

1. Rutenettceller der befolkningen er **mindre eller lik 50** 

2. *og* Rutenettceller der befolkningen er **større eller lik 15** 

La oss først se hvordan vi klassifiserer en enkelt rad:

La oss nå bruke dette på vår GeoDataFrame og lagre det i en kolonne kalt `"suitable_area"`:

Ok, vi har nye verdier i `egnet` -kolonnen.

- Hvor mange polygoner passer for oss? La oss finne ut ved hjelp av en Pandas
  funksjon kalt `value_counts()` som returnerer antall forskjellige verdier i
  vår kolonne.

In [ ]:
# Få antall verdier


Ok, så det ser ut til å være 164 passende steder.

- La oss se hvor de er lokalisert:

In [ ]:
# Plott
